## This notebook preprocesses subject 8 for Question 1: Can we predict if the subject will select Gamble or Safebet *before* the button press time?

## Behavior data

In [1]:
## Explore behavior data using pandas

import pandas as pd
import os
import numpy as np
# remove bad trials

import glob
import pandas as pd
import pickle
import scipy.io as sio
from scipy.io import loadmat
import unittest


In [2]:
game_model = pd.read_csv('game model.csv')
game_model.head()

,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,10,30,1,5,Left
1,10,20,4,5,Left
2,10,25,7,8,Right
3,10,30,1,4,Left
4,10,25,6,1,Left


In [3]:
bad_trials = sio.loadmat(os.path.join('bad_trials_OFC.mat'))['bad_trials_OFC']
bad_trials.shape

(10, 200)

In [4]:
beh_dir = 'Subject_Behav_Data/'
neur_dir = 'Subject_Ephys_Data/'

s01_df = pd.read_csv(beh_dir+'s01.csv')
s01_df['subject'] = 1
s01_df['bad_trial'] = bad_trials[0]
master_df = s01_df.merge(game_model, left_index=True, right_index=True)

master_df.head()

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,bad_trial,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,0,10,30,1,5,Left
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,0,10,20,4,5,Left
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,0,10,25,7,8,Right
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,0,10,30,1,4,Left
4,5,19.707,20.472,21.519,21.550,22.066,Right,Safebet,WouldHaveLost,1,0,10,25,6,1,Left


### Master table completed without time series data

In [5]:
for i in range(2, 11):
    number = i
    if i < 10:
        str_number = '0' + str(number)
    else:
        str_number = str(number)
    tempDF = pd.read_csv(beh_dir+'s' + str_number + '.csv')
    tempDF['subject'] = i #'s' + str_number
    tempBad = bad_trials[i-1]    
    if i == 10:
        tempBad = tempBad[:138]
    tempDF['bad_trial'] = tempBad
    tempDF = tempDF.merge(game_model, left_index=True, right_index=True)
    master_df = pd.concat([master_df, tempDF], axis = 0, sort=False)

master_df.head()

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,bad_trial,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,0,10,30,1,5,Left
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,0,10,20,4,5,Left
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,0,10,25,7,8,Right
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,0,10,30,1,4,Left
4,5,19.707,20.472,21.519,21.550,22.066,Right,Safebet,WouldHaveLost,1,0,10,25,6,1,Left


In [6]:
master_df.to_csv('raw_master_df.csv')

### Only Good Trials DF:

In [7]:
#Remove only good trials

good_master_df = master_df[master_df['bad_trial']==0]
good_master_df.head()

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,bad_trial,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,0,10,30,1,5,Left
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,0,10,20,4,5,Left
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,0,10,25,7,8,Right
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,0,10,30,1,4,Left
4,5,19.707,20.472,21.519,21.550,22.066,Right,Safebet,WouldHaveLost,1,0,10,25,6,1,Left


In [8]:
good_master_df.to_csv('good_trials_master_df.csv')

### Removing timeout trials:

In [46]:
best_master_df = good_master_df[good_master_df['choice.location']!='Timeout']
best_master_df.head()

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,bad_trial,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,0,10,30,1,5,Left
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,0,10,20,4,5,Left
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,0,10,25,7,8,Right
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,0,10,30,1,4,Left
4,5,19.707,20.472,21.519,21.550,22.066,Right,Safebet,WouldHaveLost,1,0,10,25,6,1,Left


In [47]:
best_master_df.to_csv('best_trials_master_df.csv')

In [53]:
t_neur01 = loadmat(os.path.join('Subject_Ephys_Data/s01_ofc_hg_events.mat'))['buttonpress_events_hg']

#For trial = 1
t_neur01[0]

array([[-1.5872792 , 10.512837  ,  6.0531063 , -2.2811277 , -6.5084424 ],
       [-1.5010008 , 10.57096   ,  5.9458814 , -2.4486783 , -6.6157537 ],
       [-1.4076289 , 10.626976  ,  5.815247  , -2.6147811 , -6.7176    ],
       ...,
       [-3.280595  , -3.7672455 ,  0.21471119, -3.568576  , -6.1622777 ],
       [-3.5109403 , -3.7936704 ,  0.27034077, -3.8418617 , -6.3461494 ],
       [-3.7476358 , -3.78652   ,  0.3652928 , -4.104447  , -6.490105  ]],
      dtype=float32)

In [64]:
for i in range(1, 11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)
    best_master_df['Electrode' + str_number] = np.nan
    
best_master_df.head()

/anaconda3/envs/data-x/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,...,Electrode01,Electrode02,Electrode03,Electrode04,Electrode05,Electrode06,Electrode07,Electrode08,Electrode09,Electrode10
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,19.707,20.472,21.519,21.550,22.066,Right,Safebet,WouldHaveLost,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# will need a counter since round and mat index do not match up. filtering too. 
best_master_df = best_master_df.astype('object')

for i in range(1,11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)
    
    tempMat = loadmat(os.path.join('Subject_Ephys_Data/s' + str_number +'_ofc_hg_events.mat'))['buttonpress_events_hg']
    # now we need it for each trial, each electrode. 
    #first, must filter by subject number. 
    tempSubDF = best_master_df[best_master_df['subject'] == i]
    
    tempSubDF = tempSubDF.reset_index()
    tempSubDF = tempSubDF.astype('object')
    numElectrodes = tempMat.shape[2]
    
    for e in range(1,numElectrodes+1):
        for j in range(len(tempMat)): #j is the trial
            columnData = tempMat[j, :, e-1]
            if e < 10:
                str_elec = '0' + str(e)
            else:
                str_elec = str(e)
            
            tempSubDF.loc[j, 'Electrode' + str_elec] = columnData
            
  
    #next, iterate through the rows, adding each relevant list per electrode per trial # to the dataframe. 
    # just kidding, let's just make a dataframe and then set that as the column values. 
    
    #REMEMBER: electrode inedexing is not 
    
    
tempSubDF   

ValueError: Must have equal len keys and value when setting with an iterable

In [81]:
for i in range(5):
    loadmat(os.path.join('Subject_Ephys_Data/s01_ofc_hg_events.mat'))['buttonpress_events_hg'][:,:,i]

(180, 3001)

In [76]:
df = pd.DataFrame(np.zeros((4,4)))
df = df.astype('object')
df.iloc[0, 0] = [1, 2, 3]
df.iloc[1, 2] = 5

type(df.iloc[0, 0])


list

### Neural Data:

In [19]:
s01_ephys = loadmat(os.path.join('Subject_Ephys_Data/s01_ofc_hg_events.mat'))
s01_ephys['buttonpress_events_hg'].shape[2]

5

In [11]:
sub1 = best_master_df[best_master_df['subject'] == 1]
elec1 = loadmat(os.path.join('Subject_Ephys_Data/s01_ofc_hg_events.mat'))['buttonpress_events_hg']

count = 0

sub1 = sub1.astype('object')
'''for index, row in sub1.iterrows():
    row['Electrode1'] = elec1[count, :, 0]
    count += 1'''

'''for i in range(sub1.shape[0]):
    sub1.loc[i, 'Electrode01'] = elec1[i, :, 0]
   '''
sub1

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,subject,bad_trial,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,1,0,10,30,1,5,Left
1,2,9.55,10.316,11.441,11.457,11.972,Left,Gamble,Win,1,0,10,20,4,5,Left
2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,1,0,10,25,7,8,Right
3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,1,0,10,30,1,4,Left
4,5,19.707,20.472,21.519,21.55,22.066,Right,Safebet,WouldHaveLost,1,0,10,25,6,1,Left
5,6,23.082,23.847,25.144,25.175,25.691,Right,Gamble,Win,1,0,10,30,1,7,Right
6,7,26.707,27.472,28.753,28.769,29.285,Right,Gamble,Win,1,0,10,25,2,5,Right
7,8,30.3,31.066,32.566,32.597,33.113,Right,Safebet,WouldHaveLost,1,0,10,20,7,2,Left
8,9,34.128,34.894,36.66,36.675,37.191,Left,Safebet,WouldHaveLost,1,0,10,25,9,6,Right
9,10,38.207,38.972,39.785,39.8,40.316,Left,Gamble,Win,1,0,10,30,3,8,Left


In [12]:
sub1 = sub1.reset_index()


In [15]:
sub1 = sub1.astype('object')

#sub1['Electrode01'] = elec1[:, :, 0]

sub1

elec1[:, :, 0]

array([[-1.5872792 , -1.5010008 , -1.4076289 , ..., -3.280595  ,
        -3.5109403 , -3.7476358 ],
       [-7.611065  , -7.581905  , -7.537336  , ..., -6.33127   ,
        -6.5073395 , -6.6911807 ],
       [-0.62176496, -0.29704818,  0.02554655, ..., -2.722717  ,
        -2.5983346 , -2.5041602 ],
       ...,
       [ 3.2969692 ,  3.2126997 ,  3.1295617 , ...,  9.387423  ,
         9.76672   , 10.107579  ],
       [-7.4678073 , -7.2682014 , -7.0552096 , ..., -2.511761  ,
        -2.5282414 , -2.5558338 ],
       [ 2.752385  ,  2.8246467 ,  2.8881557 , ..., -4.3141403 ,
        -4.0262365 , -3.7410843 ]], dtype=float32)

In [23]:
for i in range(1, 11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)
    sub1['Electrode' + str_number] = np.nan
sub1.head()

,index,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,...,Electrode01,Electrode02,Electrode03,Electrode04,Electrode05,Electrode06,Electrode07,Electrode08,Electrode09,Electrode10
0,0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,9.55,10.316,11.441,11.457,11.972,Left,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,19.707,20.472,21.519,21.55,22.066,Right,Safebet,WouldHaveLost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,6,23.082,23.847,25.144,25.175,25.691,Right,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,7,26.707,27.472,28.753,28.769,29.285,Right,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,8,30.3,31.066,32.566,32.597,33.113,Right,Safebet,WouldHaveLost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,9,34.128,34.894,36.66,36.675,37.191,Left,Safebet,WouldHaveLost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,10,38.207,38.972,39.785,39.8,40.316,Left,Gamble,Win,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
s01_ephys

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Oct  4 10:13:38 2017',
 '__version__': '1.0',
 '__globals__': [],
 'buttonpress_events_hg': array([[[-1.58727920e+00,  1.05128374e+01,  6.05310631e+00,
          -2.28112769e+00, -6.50844240e+00],
         [-1.50100076e+00,  1.05709600e+01,  5.94588137e+00,
          -2.44867826e+00, -6.61575365e+00],
         [-1.40762889e+00,  1.06269760e+01,  5.81524706e+00,
          -2.61478114e+00, -6.71759987e+00],
         ...,
         [-3.28059506e+00, -3.76724553e+00,  2.14711189e-01,
          -3.56857610e+00, -6.16227770e+00],
         [-3.51094031e+00, -3.79367042e+00,  2.70340770e-01,
          -3.84186172e+00, -6.34614944e+00],
         [-3.74763584e+00, -3.78652000e+00,  3.65292788e-01,
          -4.10444689e+00, -6.49010515e+00]],
 
        [[-7.61106491e+00,  3.44557953e+00,  1.27283061e+00,
          -4.36622763e+00, -8.05823231e+00],
         [-7.58190489e+00,  3.37234306e+00,  1.01458442e+00,
          -4.4942

In [38]:
e = 1
for j in range(len(s01_ephys['buttonpress_events_hg'])): #j is the trial
    columnData = s01_ephys['buttonpress_events_hg'][j, :, e-1]
    if e < 10:
        str_elec = '0' + str(e)
    else:
        str_elec = str(e)
    sub1.loc[j, 'Electrode' + str_elec] = columnData
sub1

,index,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,...,Electrode01,Electrode02,Electrode03,Electrode04,Electrode05,Electrode06,Electrode07,Electrode08,Electrode09,Electrode10
0,0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,...,"[-1.5872792, -1.5010008, -1.4076289, -1.307380...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,9.55,10.316,11.441,11.457,11.972,Left,Gamble,Win,...,"[-7.611065, -7.581905, -7.537336, -7.476991, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,12.988,13.753,14.832,14.847,15.363,Left,Safebet,WouldHaveWon,...,"[-0.62176496, -0.29704818, 0.02554655, 0.34003...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,16.378,17.144,18.144,18.175,18.691,Left,Gamble,Win,...,"[-2.9411814, -2.7336376, -2.5177944, -2.294516...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,19.707,20.472,21.519,21.55,22.066,Right,Safebet,WouldHaveLost,...,"[-3.7782059, -3.9974232, -4.184071, -4.3376594...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,6,23.082,23.847,25.144,25.175,25.691,Right,Gamble,Win,...,"[-1.5899259, -1.7192518, -1.8573122, -2.001156...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,7,26.707,27.472,28.753,28.769,29.285,Right,Gamble,Win,...,"[-7.890638, -8.107466, -8.305975, -8.483218, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,8,30.3,31.066,32.566,32.597,33.113,Right,Safebet,WouldHaveLost,...,"[0.4123459, 0.29946676, 0.18367524, 0.06582514...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,9,34.128,34.894,36.66,36.675,37.191,Left,Safebet,WouldHaveLost,...,"[-9.52892, -9.566859, -9.576814, -9.559476, -9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,10,38.207,38.972,39.785,39.8,40.316,Left,Gamble,Win,...,"[0.53784674, 0.44689927, 0.36370024, 0.2875946...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
sub1.loc[0]

index                           0
round                           1
newround.time               6.191
choice.time                 6.957
buttonpress.time            8.003
conf.time                   8.019
reveal.time                 8.535
choice.location              Left
choice.class               Gamble
outcome                       Win
subject                         1
bad_trial                       0
Safe.Bet.Amount                10
Risky.Bet.Amount               30
Risky.bet.shown.number          1
Risky.bet.hidden.number         5
Risky.Side                   Left
Electrode01                   NaN
Electrode02                   NaN
Electrode03                   NaN
Electrode04                   NaN
Electrode05                   NaN
Electrode06                   NaN
Electrode07                   NaN
Electrode08                   NaN
Electrode09                   NaN
Electrode10                   NaN
Name: 0, dtype: object

In [42]:
from scipy.io import loadmat
def Parser(dir2, mat_file):
    df11 = loadmat(os.path.join(dir2, mat_file))
    eeg_data=pd.DataFrame()
    col1=[]
    for i in range(len(df11['buttonpress_events_hg'])):
        col1.append(df11['buttonpress_events_hg'][i])
#        col2.append(df11['buttonpress_window_events_hg'][i])
#        col3.append(df11['game_events_hg'][i])
#        col4.append(df11['game_window_events_hg'][i])
    eeg_data['buttonpress_events_hg']=col1
#    eeg_data['buttonpress_window_events_hg']=col2
#    eeg_data['game_events_hg']=col3
#    eeg_data['game_window_events_hg']=col4
    return eeg_data

In [61]:
dfa = Parser(neur_dir, 's01_ofc_hg_events.mat')
dfa.iloc[0][0].shape


(3001, 5)

In [54]:
# pd.DataFrame(data=dfa)
for i in dfa['buttonpress_events_hg']:
    print(i)
len(dfa['buttonpress_events_hg'])

[[-1.5872792  10.512837    6.0531063  -2.2811277  -6.5084424 ]
 [-1.5010008  10.57096     5.9458814  -2.4486783  -6.6157537 ]
 [-1.4076289  10.626976    5.815247   -2.6147811  -6.7176    ]
 ...
 [-3.280595   -3.7672455   0.21471119 -3.568576   -6.1622777 ]
 [-3.5109403  -3.7936704   0.27034077 -3.8418617  -6.3461494 ]
 [-3.7476358  -3.78652     0.3652928  -4.104447   -6.490105  ]]
[[-7.611065   3.4455795  1.2728306 -4.3662276 -8.058232 ]
 [-7.581905   3.372343   1.0145844 -4.4942136 -8.13075  ]
 [-7.537336   3.304371   0.7965161 -4.5946717 -8.177402 ]
 ...
 [-6.33127   -3.7573326 -2.4410548 -1.722341   4.3914247]
 [-6.5073395 -4.067324  -2.7948463 -2.461653   3.983303 ]
 [-6.6911807 -4.385679  -3.1506064 -3.1681182  3.558583 ]]
[[-0.62176496 -1.5356712   2.5080845   0.55539083 -1.5325896 ]
 [-0.29704818 -1.5829968   2.5553386   0.52930766 -1.4605709 ]
 [ 0.02554655 -1.6275926   2.6337452   0.51526433 -1.383964  ]
 ...
 [-2.722717   14.2364645   1.5632588   6.5214477  -5.075972  ]
 [-2.

180

In [28]:
# dictionary: {key: value} ex - key = 'neur1'; value = the mat file
neural_dict_raw = {} 
for i in range(1,11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)
    
    tempMat = loadmat(os.path.join('Subject_Ephys_Data/s' + str_number +'_ofc_hg_events.mat'))
    
    neural_dict['neur' + str_number] = tempMat
    
neural_dict_raw

{}

In [11]:
# dictionary: {key: value} ex - key = 'neur1'; value = the mat file
neural_dict = {} 
for i in range(1,11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)

    
    tempMat = loadmat(os.path.join('Subject_Ephys_Data/s' + str_number +'_ofc_hg_events.mat'))
    for key in ['__header__', '__version__', '__globals__']:
        tempMat.pop(key)
    for key, value in tempMat.items():
        tempMat[key] = value[:, 0:950, :]
    
    neural_dict['neur' + str_number] = tempMat
    
neural_dict

{'neur01': {'buttonpress_events_hg': array([[[-1.5872792e+00,  1.0512837e+01,  6.0531063e+00, -2.2811277e+00,
           -6.5084424e+00],
          [-1.5010008e+00,  1.0570960e+01,  5.9458814e+00, -2.4486783e+00,
           -6.6157537e+00],
          [-1.4076289e+00,  1.0626976e+01,  5.8152471e+00, -2.6147811e+00,
           -6.7175999e+00],
          ...,
          [-4.9783483e+00, -4.5515995e+00,  4.1039414e+00, -5.6048551e+00,
           -1.1139888e+01],
          [-4.7898788e+00, -4.4837451e+00,  4.0328560e+00, -5.7968335e+00,
           -1.1187614e+01],
          [-4.5942297e+00, -4.4225979e+00,  3.9489701e+00, -5.9637475e+00,
           -1.1215137e+01]],
  
         [[-7.6110649e+00,  3.4455795e+00,  1.2728306e+00, -4.3662276e+00,
           -8.0582323e+00],
          [-7.5819049e+00,  3.3723431e+00,  1.0145844e+00, -4.4942136e+00,
           -8.1307497e+00],
          [-7.5373359e+00,  3.3043711e+00,  7.9651612e-01, -4.5946717e+00,
           -8.1774015e+00],
          ...,
    

In [12]:
neural_dict['neur01'].keys()

dict_keys(['buttonpress_events_hg', 'buttonpress_window_events_hg', 'game_events_hg', 'game_window_events_hg'])

In [13]:
neural_dict['neur04']['game_events_hg'].shape

(108, 950, 5)

In [14]:
#Electrode 1 for ButtonPressEventsHG for Subject 1:
neural_dict['neur01']['buttonpress_events_hg'][0, :, 0]

#Electrode 2 for ButtonPressWindowEventsHG for Subject 3:
neural_dict['neur03']['buttonpress_events_hg'][0, :, 1]

array([-3.23846251e-01, -8.32856238e-01, -1.39905560e+00, -2.01027012e+00,
       -2.65377545e+00, -3.31660962e+00, -3.98588061e+00, -4.64905405e+00,
       -5.29423380e+00, -5.91039801e+00, -6.48761225e+00, -7.01720428e+00,
       -7.49188566e+00, -7.90584707e+00, -8.25480938e+00, -8.53602695e+00,
       -8.74825478e+00, -8.89169216e+00, -8.96788692e+00, -8.97961521e+00,
       -8.93074417e+00, -8.82608223e+00, -8.67121601e+00, -8.47235012e+00,
       -8.23613262e+00, -7.96949577e+00, -7.67950964e+00, -7.37322474e+00,
       -7.05754519e+00, -6.73910761e+00, -6.42417097e+00, -6.11852026e+00,
       -5.82739782e+00, -5.55541611e+00, -5.30652094e+00, -5.08393812e+00,
       -4.89014006e+00, -4.72682619e+00, -4.59490967e+00, -4.49451065e+00,
       -4.42496729e+00, -4.38484049e+00, -4.37195063e+00, -4.38340425e+00,
       -4.41564035e+00, -4.46448374e+00, -4.52521372e+00, -4.59263849e+00,
       -4.66118574e+00, -4.72499228e+00, -4.77801704e+00, -4.81415510e+00,
       -4.82735300e+00, -

## Processing?

#### First thing we want to do is only include 0-950 (pre-decision) data points for each of the time series:

In [15]:
for key in ['__header__', '__version__', '__globals__']:
    neural_dict['neur01'].pop(key)

KeyError: '__header__'

In [ ]:
for key, value in 

neural_dict = {} 
for i in range(1,11):
    if i < 10:
        str_number = '0' + str(i)
    else:
        str_number = str(i)

    neural_dict['neur' + str_number] = loadmat(os.path.join('Subject_Ephys_Data/s' + str_number +'_ofc_hg_events.mat'))

In [ ]:
sub1_dict = loadmat(os.path.join('Subject_Ephys_Data/s01_ofc_hg_events.mat'))

for key in ['__header__', '__version__', '__globals__']:
    sub1_dict.pop(key)

In [ ]:
sub1_dict['buttonpress_events_hg'].shape

In [ ]:
#for key, value in sub1_dict.items():
#    sub1_dict[key] = value[:950]

In [ ]:
len(sub1_dict['buttonpress_events_hg'][])

In [ ]:
filtered_neural_dict = {}


neural_dict['neur01']['buttonpress_events_hg'][0, :, 0][:950]

### Choice.class will be our outcome variable

In [ ]:
beh8_df.groupby('choice.class').nunique()

### Great, we have 100 trials per choice: Gamble vs Safebet.

## Feature Engineering

In [ ]:
sfreq = 1000


from scipy.io import loadmat

neur8 = loadmat(os.path.join('s08_ofc_hg_events.mat'))

In [ ]:
neur8['buttonpress_events_hg'].shape

In [ ]:
e1_data = neur8['buttonpress_events_hg'][:,:,0]
e2_data = neur8['buttonpress_events_hg'][:,:,1]
e3_data = neur8['buttonpress_events_hg'][:,:,2]
e4_data = neur8['buttonpress_events_hg'][:,:,3]
e5_data = neur8['buttonpress_events_hg'][:,:,4]
e6_data = neur8['buttonpress_events_hg'][:,:,5]
e7_data = neur8['buttonpress_events_hg'][:,:,6]
e8_data = neur8['buttonpress_events_hg'][:,:,7]
e9_data = neur8['buttonpress_events_hg'][:,:,8]
e10_data = neur8['buttonpress_events_hg'][:,:,9]


#FEATURE 1: Min of Waves    
beh8_df['min_buttonpress_events_hg_e1'] = [min(i) for i in e1_data]
beh8_df['min_buttonpress_events_hg_e2'] = [min(i) for i in e2_data]
beh8_df['min_buttonpress_events_hg_e3'] = [min(i) for i in e3_data]
beh8_df['min_buttonpress_events_hg_e4'] = [min(i) for i in e4_data]
beh8_df['min_buttonpress_events_hg_e5'] = [min(i) for i in e5_data]
beh8_df['min_buttonpress_events_hg_e6'] = [min(i) for i in e6_data]
beh8_df['min_buttonpress_events_hg_e7'] = [min(i) for i in e7_data]
beh8_df['min_buttonpress_events_hg_e8'] = [min(i) for i in e8_data]
beh8_df['min_buttonpress_events_hg_e9'] = [min(i) for i in e9_data]
beh8_df['min_buttonpress_events_hg_e10'] = [min(i) for i in e10_data]

#FEATURE 2: Max of Waves    
beh8_df['max_buttonpress_events_hg_e1'] = [max(i) for i in e1_data]
beh8_df['max_buttonpress_events_hg_e2'] = [max(i) for i in e2_data]
beh8_df['max_buttonpress_events_hg_e3'] = [max(i) for i in e3_data]
beh8_df['max_buttonpress_events_hg_e4'] = [max(i) for i in e4_data]
beh8_df['max_buttonpress_events_hg_e5'] = [max(i) for i in e5_data]
beh8_df['max_buttonpress_events_hg_e6'] = [max(i) for i in e6_data]
beh8_df['max_buttonpress_events_hg_e7'] = [max(i) for i in e7_data]
beh8_df['max_buttonpress_events_hg_e8'] = [max(i) for i in e8_data]
beh8_df['max_buttonpress_events_hg_e9'] = [max(i) for i in e9_data]
beh8_df['max_buttonpress_events_hg_e10'] = [max(i) for i in e10_data]


#FEATURE 3: Average of Waves    
beh8_df['avg_buttonpress_events_hg_e1'] = [np.mean(i) for i in e1_data]
beh8_df['avg_buttonpress_events_hg_e2'] = [np.mean(i) for i in e2_data]
beh8_df['avg_buttonpress_events_hg_e3'] = [np.mean(i) for i in e3_data]
beh8_df['avg_buttonpress_events_hg_e4'] = [np.mean(i) for i in e4_data]
beh8_df['avg_buttonpress_events_hg_e5'] = [np.mean(i) for i in e5_data]
beh8_df['avg_buttonpress_events_hg_e6'] = [np.mean(i) for i in e6_data]
beh8_df['avg_buttonpress_events_hg_e7'] = [np.mean(i) for i in e7_data]
beh8_df['avg_buttonpress_events_hg_e8'] = [np.mean(i) for i in e8_data]
beh8_df['avg_buttonpress_events_hg_e9'] = [np.mean(i) for i in e9_data]
beh8_df['avg_buttonpress_events_hg_e10'] = [np.mean(i) for i in e10_data]

#FEATURE 4: Average of Waves    
beh8_df['std_buttonpress_events_hg_e1'] = [np.std(i) for i in e1_data]
beh8_df['std_buttonpress_events_hg_e2'] = [np.std(i) for i in e2_data]
beh8_df['std_buttonpress_events_hg_e3'] = [np.std(i) for i in e3_data]
beh8_df['std_buttonpress_events_hg_e4'] = [np.std(i) for i in e4_data]
beh8_df['std_buttonpress_events_hg_e5'] = [np.std(i) for i in e5_data]
beh8_df['std_buttonpress_events_hg_e6'] = [np.std(i) for i in e6_data]
beh8_df['std_buttonpress_events_hg_e7'] = [np.std(i) for i in e7_data]
beh8_df['std_buttonpress_events_hg_e8'] = [np.std(i) for i in e8_data]
beh8_df['std_buttonpress_events_hg_e9'] = [np.std(i) for i in e9_data]
beh8_df['std_buttonpress_events_hg_e10'] = [np.std(i) for i in e10_data]

beh8_df.head()

In [ ]:
X=beh8_df.iloc[:,9:]
X.head()

In [ ]:
Y=beh8_df['choice.class']
print (Y.value_counts())  #BALANCED DATASET, GOOD!

In [ ]:
print("Feature vector shape=", X.shape)
print("Class shape=", Y.shape)

## MODELING

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in test data:',len(x_test))

In [ ]:
from sklearn import linear_model

# Name our logistic regression object
LogisticRegressionModel = linear_model.LogisticRegression()

# we create an instance of logistic Regression Classifier and fit the data.
print ('Training a logistic Regression Model..')
LogisticRegressionModel.fit(x_train, y_train)

In [ ]:
training_accuracy=LogisticRegressionModel.score(x_train,y_train)
print ('Training Accuracy:',training_accuracy)

In [ ]:
test_accuracy=LogisticRegressionModel.score(x_test,y_test)
print('Accuracy of the model on unseen test data: ',test_accuracy)

In [ ]:
#CONFUSION MATRIX

from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = LogisticRegressionModel.predict(x_test)
ConfusionMatrix=pd.DataFrame(confusion_matrix(y_true, y_pred),columns=['Predicted Gamble','Predicted Safe'],index=['Actual Gamble','Actual Safe'])
print ('Confusion matrix of test data is: \n',ConfusionMatrix)

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import precision_score
print("Average precision for the 2 classes is - ", precision_score(y_true, y_pred, average = None) )

In [ ]:
from sklearn.metrics import recall_score
print("Average recall for the 2 classes is - ", recall_score(y_true, y_pred, average = None) )

In [ ]:
perceptron = Perceptron(max_iter=100)
perceptron.fit(x_train, y_train)
perceptron_train_acc = perceptron.score(x_train, y_train)
perceptron_test_acc = perceptron.score(x_test, y_test)
print ('perceptron training acuracy= ',perceptron_train_acc)
print('perceptron test accuracy= ',perceptron_test_acc)

In [ ]:
# Adaboost
adaboost = AdaBoostClassifier()
adaboost.fit(x_train, y_train)
adaboost_train_acc = adaboost.score(x_train, y_train)
adaboost_test_acc = adaboost.score(x_test, y_test)
print ('adaboost training acuracy= ',adaboost_train_acc)
print('adaboost test accuracy= ',adaboost_test_acc)

In [ ]:
# Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)
random_forest_train_acc = random_forest.score(x_train, y_train)
random_forest_test_acc = random_forest.score(x_test, y_test)
print('random_forest training acuracy= ',random_forest_train_acc)
print('random_forest test accuracy= ',random_forest_test_acc)

# Further work: 

## We need to run k-fold validation. Too small a dataset. 

### Can potentially remove 0,1,2 number and 8,9,10 number 